In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from datetime import date

import statsmodels.api as sm

#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LinearRegression
#from sklearn import metrics

#import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from nltk.util import ngrams
from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
#SET OUT THE QUESTIONS OF INTEREST
"""
1) Is there a difference in the average price per night of Airbnb's in Seattle and Boston?
2) What are people saying about their stays in Boston and Seattle?
3) When it comes to price, is it really all about location, location, location? Are downtown properties that much more expensive than those further out?
4) What are the most important factors in deciding the price per night of an Airbnb and is city one of them?

"""

In [ ]:
# INTRODUCE THE DATA

In [ ]:
# You will need to edit these steps for your own machine

Sea_calendar = pd.read_csv('D:\\lori_python\\UDACITY\\Lesson 1 Intro to DS\\Project 1 blog post\\Data\\Seattle data\\calendar.csv')
Sea_listings = pd.read_csv('D:\\lori_python\\UDACITY\\Lesson 1 Intro to DS\\Project 1 blog post\\Data\\Seattle data\\listings.csv')
Sea_reviews = pd.read_csv('D:\\lori_python\\UDACITY\\Lesson 1 Intro to DS\\Project 1 blog post\\Data\\Seattle data\\reviews.csv')

In [ ]:
#See comment above
Bos_calendar = pd.read_csv('D:\\lori_python\\UDACITY\\Lesson 1 Intro to DS\\Project 1 blog post\\Data\\Boston data\\calendar.csv')
Bos_listings = pd.read_csv('D:\\lori_python\\UDACITY\\Lesson 1 Intro to DS\\Project 1 blog post\\Data\\Boston data\\listings.csv')
Bos_reviews = pd.read_csv('D:\\lori_python\\UDACITY\\Lesson 1 Intro to DS\\Project 1 blog post\\Data\\Boston data\\reviews.csv')

In [ ]:
pd.set_option('display.max_columns', 200)
Sea_listings.head()

In [ ]:
Sea_listings.columns

In [ ]:
Bos_listings.head()

In [ ]:
# Let's get some idea of when the data is from and whether the different data sets all refer tyo sme pool of properties

Sea_listings['last_scraped'].value_counts()

Bos_listings['last_scraped'].value_counts()

In [ ]:

Sea_reviews['date'].min()

Sea_reviews['date'].max()

In [ ]:
Bos_reviews['date'].min()

Bos_reviews['date'].max()

In [ ]:
#Let's check that the properties in the reviews data sets are the same as in the listings data
Sea_ser1 = Sea_reviews['listing_id']
Sea_ser2 = Sea_listings['id']
Intersect = set(Sea_ser1) & set(Sea_ser2)
print(len(Intersect))

Sea_reviews['listing_id'].nunique() # From this we can conclude that all Seattle reviews relate to properties in the listings data set


In [ ]:
Bos_ser1 = Bos_reviews['listing_id']
Bos_ser2 = Bos_listings['id']
BIntersect = set(Bos_ser1) & set(Bos_ser2)
print(len(BIntersect))

Bos_reviews['listing_id'].nunique() # Again can conclude properties are the same in the two data sets

In [ ]:
# INTRODUCE Q1

In [ ]:
Sea_listings['price'] = Sea_listings['price'].str.replace(',', '')

Sea_listings['price'] = Sea_listings['price'].str.replace('$', '')

Sea_listings['price'] = Sea_listings['price'].str.replace(' ', '')

Sea_listings['price'].head()

Sea_listings['price'] = Sea_listings['price'].astype('float')

In [ ]:
(np.sum(Sea_listings['price'].isnull()) / len(Sea_listings['price'])) * 100 # % missing values

In [ ]:
Sea_listings['price'].min()

In [ ]:
Sea_listings['price'].max()

In [ ]:
Sea_listings['price'].mean()

In [ ]:
Sea_listings['price_range($)'] = Sea_listings['price'].apply(lambda x:'<50' if x < 50 else '50-100' if x <100 else '100-150' if x <150 
                                                 else '150-200' if x < 200 else '200-250' if x < 250 else '250-300' if x <300
                                                     else '>300')

In [ ]:
Sea_listings['price_range($)'].value_counts()

In [ ]:
"""
legend_map = {0.0: '<$50',
              50.0: '$50-$100',
             100.0: '$100-$150',
             150.0: '$150-$200',
             200.0: '$200-$250',
             250.0: '$250-$300',
             300.0: '>$300'}
"""

In [ ]:
plt.figure(figsize=(16,12))
sns.scatterplot(x='longitude', y='latitude', data=Sea_listings,
                hue='price_range($)', hue_order = ['<50', '50-100', '100-150', '150-200', '200-250', '250-300', '>300'], 
                palette='viridis').set(title="Map of Seattle Airbnb listings")
#plt.savefig('Seattle_Map2.png')

In [ ]:
# Let's take the count for each neighbourhood to help with interpreting the map

Sea_listings['neighbourhood'].value_counts()

In [ ]:
# Boston map

In [ ]:
Bos_listings['price'] = Bos_listings['price'].str.replace(',', '')

Bos_listings['price'] = Bos_listings['price'].str.replace('$', '')

Bos_listings['price'] = Bos_listings['price'].str.replace(' ', '')

Bos_listings['price'].head()

Bos_listings['price'] = Bos_listings['price'].astype('float')

In [ ]:
(np.sum(Bos_listings['price'].isnull()) / len(Bos_listings['price'])) * 100 # % missing values

In [ ]:
Bos_listings['price'].min()

In [ ]:
Bos_listings['price'].max()

In [ ]:
Bos_listings['price'].mean()

In [ ]:
Bos_listings['price_range($)'] = Bos_listings['price'].apply(lambda x:'<50' if x < 50 else '50-100' if x <100 else '100-150' if x <150 
                                                 else '150-200' if x < 200 else '200-250' if x < 250 else '250-300' if x <300
                                                     else '>300')

In [ ]:
plt.figure(figsize=(16,12))
sns.scatterplot(x='longitude', y='latitude', data=Bos_listings,
                hue='price_range($)', hue_order = ['<50', '50-100', '100-150', '150-200', '200-250', '250-300', '>300'], 
                palette='viridis').set(title="Map of Seattle Airbnb listings")
# plt.savefig('Boston_Map6.png')

In [ ]:
Bos_listings['neighbourhood'].value_counts()

In [ ]:
#In Seattle the downtown doesn't command the same premium as in Boston as the lighter dots are spread out, whereas in Boston anything that isn't central is a darker colour

In [ ]:
#Q2. what are people saying about the properties in each city?

In [ ]:
len(Sea_reviews)

In [ ]:
len(Bos_reviews)

In [ ]:
np.sum(Sea_reviews.isnull())

In [ ]:
np.sum(Bos_reviews.isnull())

In [ ]:
Sea_reviews = Sea_reviews.dropna()
Bos_reviews= Bos_reviews.dropna()

In [ ]:
#When you look at reviews you often find that there are automated posting when a reservaton is cancelled. Let's remove these
substring = ["automated posting"]
Sea_reviews = Sea_reviews[~Sea_reviews.comments.str.contains('|'.join(substring))]
print(len(Sea_reviews))

In [ ]:
Bos_reviews = Bos_reviews[~Bos_reviews.comments.str.contains('|'.join(substring))]
print(len(Bos_reviews))

In [ ]:
Sea_reviews.head()

In [ ]:
Bos_reviews.head()

In [ ]:
Sea_reviews['comments'] = Sea_reviews['comments'].str.lower()
Bos_reviews['comments'] = Bos_reviews['comments'].str.lower()

In [ ]:
""" This function takes a text input and prepares it for analysis by creating a token for each element, removing common words, lemmatizing the tokens ans then recombining them into one string per row """

def clean_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    
    # Join the tokens back into a string
    clean_text = ' '.join(lemmatized_tokens)
    
    return clean_text

In [ ]:
#Preprocessing

In [ ]:
Cleaned_SText = Sea_reviews['comments'].apply(clean_text)
S_reviews = pd.DataFrame({'cleaned text': Cleaned_SText})

S_reviews.head()

In [ ]:
Cleaned_BText = Bos_reviews['comments'].apply(clean_text)
B_reviews = pd.DataFrame({'cleaned text': Cleaned_BText})


In [ ]:
#Find most common words in reviews

In [ ]:
#Analyse sentiment # PLUS THERE WILL BE DUPLICATE CODE UNDER HERE

In [ ]:
sio = SentimentIntensityAnalyzer()

In [ ]:
Sea_reviews['scores'] = Sea_reviews['rejoined'].apply(lambda review: sio.polarity_scores(review))
Sea_reviews['compound']  = Sea_reviews['scores'].apply(lambda score_dict: score_dict['compound'])
Sea_reviews['comp_score'] = Sea_reviews['compound'].apply(lambda c: 'pos' if c >0 else 'neut' if c == 0 else 'neg')

In [ ]:
Sea_reviews['comp_score'].value_counts()

In [ ]:
S_Percent_Revs = (Sea_reviews['comp_score'].value_counts() / len(Sea_reviews['comp_score'])) *100

In [ ]:
S_Percent_Revs 

In [ ]:
# Look at most frequent words and phrases

In [ ]:
All_S_Words = ' '.join([word for word in Sea_reviews['rejoined']])

In [ ]:
All_S_Words

In [ ]:
S_words = nltk.word_tokenize(All_S_Words)
S_fd = FreqDist(S_words)

In [ ]:
S_fd.most_common(30)

In [ ]:
S_phrases = Counter(list(ngrams(S_words, 3)))
S_phrases.most_common(5)

In [ ]:
#Same again for Boston data

In [ ]:
Bos_reviews['tokenized'] = Bos_reviews.apply(lambda x: token(x['comments']), axis = 1) 
Bos_reviews['no_stops'] = Bos_reviews.apply(lambda x: remove_stops(x['tokenized']), axis = 1)
Bos_reviews['lemmed'] = Bos_reviews.apply(lambda x: lemmatize(x['no_stops']), axis =1) 
Bos_reviews['rejoined'] = Bos_reviews.apply(lambda x: rejoined(x['lemmed']), axis=1)

In [ ]:
Bos_reviews['scores'] = Bos_reviews['rejoined'].apply(lambda review: sio.polarity_scores(review))
Bos_reviews['compound']  = Bos_reviews['scores'].apply(lambda score_dict: score_dict['compound'])

In [ ]:
Bos_reviews['comp_score'] = Bos_reviews['compound'].apply(lambda c: 'pos' if c >0 else 'neut' if c==0 else 'neg')

In [ ]:
B_Percent_Revs = (Bos_reviews['comp_score'].value_counts() / len(Bos_reviews['comp_score'])) *100

In [ ]:
B_Percent_Revs

In [ ]:
All_B_Words = ' '.join([word for word in Bos_reviews['rejoined']])

In [ ]:
B_words = nltk.word_tokenize(All_B_Words)
B_fd = FreqDist(B_words)

In [ ]:
B_fd.most_common(30)

In [ ]:
B_Phrases = Counter(list(ngrams(B_words,3)))
B_Phrases.most_common(5)

In [ ]:
#  Q3 STARTS HERE - What factors determine the price of stays, is there a difference between the two cities?

In [ ]:
Sea_listings.shape

In [ ]:
Bos_listings.shape

In [ ]:
a = np.setdiff1d(Sea_listings.columns, Bos_listings.columns)
print (a) # Columns that are in Seattle data but not Boston

In [ ]:
b = np.setdiff1d(Bos_listings.columns, Sea_listings.columns)
print (b) # Columns that are in Boston data but not Seattle

In [ ]:
Bos_listings = Bos_listings.drop(['access', 'house_rules', 'interaction'], axis=1)

In [ ]:
Bos_listings.shape

In [ ]:
Sea_listings['Boston_City'] = 0

In [ ]:
Sea_listings['Boston_City'].value_counts()

In [ ]:
Bos_listings['Boston_City'] = 1

In [ ]:
Bos_listings['Boston_City'].value_counts()

In [ ]:
listings = Sea_listings.append(Bos_listings)

In [ ]:
listings.shape

In [ ]:
listings.info() #Why is licence in here? Most variable have around 7k observations.
#Let's pick variables of interest out from the below for modelling, as not all of these are relevant to proce

In [ ]:
listings = listings[['price', 'host_is_superhost', 'room_type', 'bed_type', 'experiences_offered', 'host_has_profile_pic', 'Boston_City', 'is_location_exact', 'property_type','bathrooms', 'bedrooms', 'beds', 'security_deposit', 'cleaning_fee', 'minimum_nights', 'maximum_nights', 'availability_30', 'number_of_reviews', 'review_scores_rating', 'instant_bookable', 'cancellation_policy', 'amenities' ]]

In [ ]:
listings.info()

In [ ]:
listings.head()

In [ ]:
listings['experiences_offered'].value_counts() # looks like a dudd

In [ ]:
listings = listings.drop(['experiences_offered'], axis=1)

In [ ]:
#Let's start tidying up the data.
# Have binary variables that need to be 1/0, text that needs to be a scale, punctuation to remove and missing values to impute.

In [ ]:
ammen = listings['amenities'].str.split(",")

ammen

flat_list = []
for sublist in ammen:
    for item in sublist:
        flat_list.append(item)

flat_list

char = '{'
char2 ='}'
char3 = '"'
char4 = "'"

for idx, ele in enumerate(flat_list):
        flat_list[idx] = ele.replace(char, '').replace(char2, "").replace(char3,'').replace(char4,'')

flat_list

unique_amens = set(flat_list)

unique_amens

unique_amens.remove('translation missing: en.hosting_amenity_49')

unique_amens.remove('translation missing: en.hosting_amenity_50')

unique_amens.remove('')

unique_amens

test = listings.amenities.str.get_dummies(sep = ",")

#test

test.columns = test.columns.astype(str).str.replace(r'[""]', "")

test

amen_dums = test.drop(columns=[col for col in test.columns if col not in unique_amens])

amen_dums

In [ ]:
listings = listings.join(amen_dums)

In [ ]:
listings = listings.replace(',','', regex=True)

In [ ]:
listings = listings.replace('&','', regex=True)

In [ ]:
listings = listings.replace('$','', regex=True)

In [ ]:
def binary(col): 
    return col.map(dict(t=1, f=0))

In [ ]:
listings[['instant_bookable', 'host_is_superhost', 'host_has_profile_pic', 'is_location_exact']]=listings[['instant_bookable', 'host_is_superhost', 'host_has_profile_pic',  'is_location_exact']].apply(binary)

In [ ]:
listings.head()

In [ ]:
listings['property_type'].value_counts()

In [ ]:
listings['property_type'] = listings['property_type'].apply(lambda x: 2 if x =='House' else 2 if x == 'Townhouse' else 1 if
                                                           x ==' Apartment' else 1 if x=='Condominium' else 0)

In [ ]:
listings['cancellation_policy'].value_counts()

In [ ]:
listings['cancellation_policy'] = listings['cancellation_policy'].apply(lambda x: 3 if x =='super_strict_30' else 2 if x =='strict'
                                                        else 1 if x=='moderate' else 0)

In [ ]:
listings.head()

In [ ]:
listings['room_type'].value_counts()

In [ ]:
room = pd.get_dummies(listings['room_type'], prefix='room', drop_first=True)

In [ ]:
listings = listings.join(room)

In [ ]:
listings['bed_type'].value_counts()

In [ ]:
bed = pd.get_dummies(listings['bed_type'], prefix ='bed', drop_first=True)

In [ ]:
listings = listings.join(bed)

In [ ]:
listings.head()

In [ ]:
listings.info()

In [ ]:
listings['cleaning_fee'].value_counts()

In [ ]:
#Security deposit and cleaning fee still have dollar signs. Need to be converted to numeric type before missinga data can be filled

In [ ]:
listings['security_deposit']= listings['security_deposit'].str.replace('$','', regex=True)

In [ ]:
listings['cleaning_fee']= listings['cleaning_fee'].str.replace('$','', regex=True)

In [ ]:
listings['security_deposit'] = listings['security_deposit'].astype('float')

In [ ]:
listings['cleaning_fee'] = listings['cleaning_fee'].astype('float')

In [ ]:
#These will feed into a function that imputes missing data using average grouped by related values
impute_mean_deposit = listings.groupby('bedrooms', dropna=False).mean()['security_deposit'] 
impute_mean_cleaning = listings.groupby('bedrooms', dropna=False).mean()['cleaning_fee']

In [ ]:
def impute_deposit (bedrooms,fee):
    if np.isnan(fee):
        return impute_mean_deposit[bedrooms]
    else:
        return fee

In [ ]:
listings['security_deposit'] = listings.apply(lambda x: impute_deposit(x['bedrooms'], x['security_deposit']), axis=1)

In [ ]:
def impute_cleaning (bedrooms,fee):
    if np.isnan(fee):
        return impute_mean_cleaning[bedrooms]
    else:
        return fee

In [ ]:
listings['cleaning_fee'] = listings.apply(lambda x: impute_cleaning(x['bedrooms'], x['cleaning_fee']), axis=1)

In [ ]:
listings.head()

In [ ]:
listings.info()

In [ ]:
listings['price'] = listings['price'].astype('float')

In [ ]:
listings['price'] = listings['price'].astype('Int64')

In [ ]:
listings = listings.drop(['room_type', 'bed_type', 'amenities'], axis=1)

In [ ]:
listings = listings.dropna()

In [ ]:
Y = listings['price']
X = listings.drop(['price'], axis=1)
X = sm.add_constant(X)
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 101)

#model1 = LinearRegression()    #This is sklearn approach
#model1.fit(X_train, Y_train)

model = sm.OLS(Y.astype('float'), X.astype('float')).fit()

In [ ]:
print(model.summary())